# Importing important libreries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for visualisation

import tensorflow as tf # for deep nn models
import scikitplot as skplt # for visualisation
from sklearn import metrics 


# Making a class for data importing

The data has been saved in npy format, so in order to import it, we will make a new class, that will do it for us.

In [3]:
class read_data:
    """After calling this class, we will have our data loaded"""
    
    def __init__(self):
        
        train_path = '/kaggle/input/bioinformatics/machine_learning/JUND_TF/train/'
        valid_path = '/kaggle/input/bioinformatics/machine_learning/JUND_TF/valid/'
        test_path = '/kaggle/input/bioinformatics/machine_learning/JUND_TF/test/'
        
        self.train = {}; self.valid = {}; self.test = {} # making empty dictionaries 
        
        # Importing Training dataset: witch consists of features X, wights W and labels Y       
        with open(train_path + 'train_X.npy', 'rb') as f:
            self.train['X'] = np.load(f) # as it is in .npy format
        f.close()
        
        with open(train_path + 'train_y.npy', 'rb') as f:
            self.train['y'] = np.load(f)
        f.close()
        
        with open(train_path + 'train_w.npy', 'rb') as f:
            self.train['w'] = np.load(f)
        f.close()
        
        with open(train_path + 'train_ids.npy', 'rb') as f:
            self.train['ids'] = np.load(f, allow_pickle=True)
        f.close()
        
        
        # All the same for the VALIDATION dataset
        with open(valid_path + 'valid_X.npy', 'rb') as f:
            self.valid['X'] = np.load(f)
        f.close()
        
        with open(valid_path + 'valid_y.npy','rb') as f:
            self.valid['y'] = np.load(f)
        f.close()
        
        with open(valid_path + 'valid_w.npy', 'rb') as f:
            self.valid['w'] = np.load(f)
        f.close()
        
        with open(valid_path + 'valid_ids.npy', 'rb') as f:
            self.valid['ids'] = np.load(f, allow_pickle=True)
        f.close()
        
        # All the same for the TEST dataset
        with open(test_path+'test_X.npy','rb') as f:
            self.test['X'] = np.load(f)
        f.close()
        
        with open(test_path+'test_y.npy','rb') as f:
            self.test['y'] = np.load(f)
        f.close()
        
        with open(test_path+'test_w.npy','rb') as f:
            self.test['w'] = np.load(f)
        f.close()
        
        with open(test_path+'test_ids.npy','rb') as f:
            self.test['ids'] = np.load(f,allow_pickle=True)
        f.close()
        
        # Adding chromatic accessibility fearure from accessability.txt
        def add_chrom(self):
            
            span_accessibility = {}
            for line in open ('/kaggle/input/bioinformatics/machine_learning/JUND_TF/accessibility.txt'):
                fields = line.split() # splitting features so we get KEY and VALUE
                span_accessibility[fields[0]] = float(fields[1])
        
    
            # Finding the correct value for txt 
            def dicarray_df(case):

                # Making sure that we take the right set, before sorting
                if (case == 'train'):
                    case_id = self.train
                elif(case == 'valid'):
                    case_id = self.valid
                elif(case == 'test'):
                    case_id = self.test

                # Making temporary empty DataFrame, for sorting
                ldf = pd.DataFrame([k,*v] for k,v in case_id.items())

                ldf = ldf.T 
                ldf.columns = ['X', 'y', 'w', 'ids']
                ldf.drop(0, axis = 0, inplace = True) # allows you to remove row (axis = 0) number 0 (1st one)
                ldf.reset_index(drop = True, inplace = True) # removing current indexes 

                # Extracting chromosom
                ldf['chrom'] = ldf['ids'].map(span_accessibility) # map puts together 'ids' with span_acceibility 
                # print(os.getcwd())
                # os.chdir('/kaggle/working/')

                # This function adds chromosom info and saves it 
                np.save(f'./chrom_{case}.npy', ldf['chrom'].values)
            
            # calling function
            dicarray_df('train')
            dicarray_df('valid')
            dicarray_df('test')

            # Match and save
            array = np.load('./chrom_train.npy')
            self.train['chrom'] = array

            array = np.load('./chrom_valid.npy')
            self.valid['chrom'] = array

            array = np.load('./chrom_test.npy')
            self.test['chrom'] = array

# Importing data 

Here are presented Encoded data of DNA sequence from the chromosome 22. The data thath will be used is from the course: Deep Learning for the Life Sciences. The aim of this project is to predict Transcription Factor binding locations in ch22. The data is presented in the form of one dimensional data, where the DNA sequence of ch22, which is consisted od more than 50 milion bases, is split in sequences of length 101 bases. Those sequences are then encoded, using One Hot Encoding, where each base is presented with the vector of lenght 4. The encodings are: 
А → [1, 0, 0, 0],
C → [0, 1, 0, 0],
G → [0, 0, 1, 0],
T → [0, 0, 0, 1],

In [4]:
data = read_data()

In [ ]:
# # Checking the shape of loaded data
data_info = {"Features": ["Feature Matrix", "Labels", "Sample Weighting"],
             "Train": [data.train['X'].shape, data.train['y'].shape,data.train['w'].shape],
            "Validation": [data.valid['X'].shape, data.valid['y'].shape, data.valid['w'].shape],
            "Test": [data.test['X'].shape,data.test['y'].shape, data.test['w'].shape]}
data_info = pd.DataFrame(data_info)

data_info

In [ ]:
# Number of positives and negatives in our dataset
labels = pd.DataFrame(np.concatenate([data.train['y'], data.test['y'], data.valid['y']], axis = 0), columns = ['y'])
labels['y'].value_counts()

In [ ]:
trening_labels = pd.DataFrame(np.concatenate([data.train['y']]), columns = ['y'])

test_labels = pd.DataFrame(np.concatenate([data.test['y']]), columns = ['y'])

valid_labels = pd.DataFrame(np.concatenate([data.valid['y']]), columns = ['y'])

print(f"Trening set:\n {trening_labels['y'].value_counts()}")
print(f"\nValidation set:\n {valid_labels['y'].value_counts()}" )
print(f"\nTest set:\n {test_labels['y'].value_counts()}" )


# Balancing classes

**FIRST WE MAKE A DATAFRAME OUT OF OUR DATA**

In [5]:
# Making a DataFrame from the train samples
# for undersampling
x = data.train["X"]
y = data.train['y']
w = data.train['w']

df = {'X': [], 'y': [], 'w': []}
for i in range(x.shape[0]):
    df['X'].append(x[i])
    df['y'].append(y[i])
    df['w'].append(w[i])

train_df = pd.DataFrame(df)
train_df.head


<bound method NDFrame.head of                                                         X    y  \
0       [[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [1,...  [0]   
1       [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,...  [0]   
2       [[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...  [0]   
3       [[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...  [0]   
4       [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [0,...  [0]   
...                                                   ...  ...   
276211  [[0, 0, 1, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1,...  [0]   
276212  [[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0,...  [0]   
276213  [[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,...  [0]   
276214  [[0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0,...  [0]   
276215  [[0, 0, 1, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0,...  [0]   

                           w  
0       [0.5021232657572496]  
1       [0.5021232657572496]  
2       [0.5021232657572496]  
3       [0.5021232657572496]  
4       [0.50212326575

**DOWNSAMPLING DATA**
TAKING 50 TIMES MORE NEGATIVES THAN POSITIVES

In [6]:
# DOWNSAMPLING

train_positives = train_df[train_df['y'] == 1]
train_negatives = train_df[train_df['y'] == 0]

# We need to sample part of the negative ones, since there are too many negative samples
np.random.seed(1234) # MAKING SURE OUR SAMPLED DATA IS ALWAYS THE SAME
train_negatives_downsampled = train_negatives.sample( n = train_positives.shape[0]*50)

data_train_downsampled = pd.concat([train_positives, train_negatives_downsampled])

data_train_downsampled.shape, data_train_downsampled.head()


((59568, 3),
                                                       X    y  \
 718   [[0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [0,...  [1]   
 809   [[0, 0, 1, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0,...  [1]   
 997   [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [1,...  [1]   
 1261  [[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [0,...  [1]   
 1393  [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [1,...  [1]   
 
                         w  
 718   [118.2431506849315]  
 809   [118.2431506849315]  
 997   [118.2431506849315]  
 1261  [118.2431506849315]  
 1393  [118.2431506849315]  )

In [ ]:
print("downsampled negatives> ",data_train_downsampled[data_train_downsampled['y'] == 0].shape[0])
print("positives> ", data_train_downsampled[data_train_downsampled['y'] == 1].shape[0])

**TAKING OUR DATA INTO IT'S FIRST SHAPE**

In [7]:
X_train = np.concatenate(np.array(data_train_downsampled['X'])).reshape(data_train_downsampled['X'].shape[0],101,4)
print("Shape of the features: ",X_train.shape)

y_train = np.concatenate(np.array(data_train_downsampled['y'])).reshape(data_train_downsampled['y'].shape[0],1)
print("Shape of lables", y_train.shape)

w_train = np.concatenate(np.array(data_train_downsampled['w'])).reshape(data_train_downsampled['w'].shape[0],1)

Shape of the features:  (59568, 101, 4)
Shape of lables (59568, 1)


# Defining important features

In [8]:
METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.AUC(name='auc')]

BATCH_SIZE = 512

# Input_Shape = data.train['X'].shape[1:]
Input_Shape = X_train.shape[1:]

# MAKING MODELS

# 1D CNN= book

In [14]:
model_book = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters = 90, 
                           kernel_size = 10,
                          activation = 'relu',
                          padding = 'same',
                          input_shape = Input_Shape),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Conv1D(filters = 90, 
                           kernel_size = 10,
                          activation = 'relu',
                          padding = 'same'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Conv1D(filters = 90, 
                           kernel_size = 10,
                          activation = 'relu',
                          padding = 'same'),
        tf.keras.layers.Dropout(0.5),
    
        tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [15]:
model_book.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3),
               loss = 'binary_crossentropy',
               metrics = METRICS)

model_book.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 101, 90)           3690      
                                                                 
 dropout_3 (Dropout)         (None, 101, 90)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 101, 90)           81090     
                                                                 
 dropout_4 (Dropout)         (None, 101, 90)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 101, 90)           81090     
                                                                 
 dropout_5 (Dropout)         (None, 101, 90)           0         
                                                                 
 flatten_1 (Flatten)         (None, 9090)             

In [16]:
np.random.seed(123)
model_book.fit(X_train, y_train,
            validation_data=(data.valid['X'], data.valid['y']),
               sample_weight = w_train,
            epochs = 50, batch_size = BATCH_SIZE)

Epoch 1/50
117/117 [==============================] - 109s 924ms/step - loss: 1.3423 - auc: 0.6550 - val_loss: 1.4823 - val_auc: 0.5761
Epoch 2/50
117/117 [==============================] - 108s 925ms/step - loss: 1.3045 - auc: 0.5398 - val_loss: 1.0546 - val_auc: 0.6176
Epoch 3/50
117/117 [==============================] - 108s 925ms/step - loss: 1.3058 - auc: 0.5584 - val_loss: 1.3488 - val_auc: 0.6595
Epoch 4/50
117/117 [==============================] - 109s 929ms/step - loss: 1.2728 - auc: 0.6093 - val_loss: 1.3571 - val_auc: 0.6707
Epoch 5/50
117/117 [==============================] - 110s 944ms/step - loss: 1.2353 - auc: 0.6674 - val_loss: 1.2034 - val_auc: 0.6985
Epoch 6/50
117/117 [==============================] - 118s 1s/step - loss: 1.1950 - auc: 0.7065 - val_loss: 1.1674 - val_auc: 0.7156
Epoch 7/50
117/117 [==============================] - 110s 941ms/step - loss: 1.1637 - auc: 0.7289 - val_loss: 1.0281 - val_auc: 0.7423
Epoch 8/50
117/117 [==============================]

In [17]:
model_book.evaluate(x = data.test['X'], y = data.test['y'])


1079/1079 [==============================] - 18s 17ms/step - loss: 0.1376 - auc: 0.7446


[0.13763456046581268, 0.7445834279060364]

# 1D CNN model

In [ ]:
model_cnn = tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(256, 
                                kernel_size = 3,
                                padding = "valid", 
                                activation = "relu",
                                input_shape=Input_Shape),    
            tf.keras.layers.MaxPooling1D(3),
            tf.keras.layers.Conv1D(256, 
                                kernel_size = 3,
                                padding = "valid", 
                                activation = "relu"),    
            tf.keras.layers.MaxPooling1D(3),
            tf.keras.layers.Dropout(0.5),



    
            tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(128, activation = 'relu'),
#             tf.keras.layers.Dense(64, activation = 'relu'),
#             tf.keras.layers.Dense(64),
#             tf.keras.layers.Dropout(0.2),
# #             tf.keras.layers.Dense(32),
#             tf.keras.layers.Dense(16),
            tf.keras.layers.Dense(1,activation='sigmoid') # ISPROBAJ OPET SA DOWNSAMPLING 50
])

In [ ]:
opt_cnn = tf.keras.optimizers.Adam(learning_rate = 1e-3)
model_cnn.compile(optimizer = opt_cnn,
                 loss = 'binary_crossentropy',
                 metrics = METRICS)
model_cnn.summary()

In [ ]:
np.random.seed(1234)
model_cnn.fit(x = data.train['X'], y= data.train['y'], 
              validation_data = (data.valid['X'], data.valid['y']),
              sample_weight = data.train['w'],
             epochs = 10, batch_size = 512)

In [ ]:
model_cnn.evaluate(data.test['X'],data.test['y'])

# 1D CNN + Bi-LSTM

In [ ]:
model_cnn_bilstm = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, 
                          kernel_size = 3, 
                          input_shape = Input_Shape),
    tf.keras.layers.MaxPooling1D(3),
#     tf.keras.layers.Conv1D(256, 
#                           kernel_size = 3),
#     tf.keras.layers.MaxPooling1D(3),
    tf.keras.layers.Dropout(0.25), 
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128),
#     tf.keras.layers.Dense(64),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model_cnn_bilstm.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3),
                        loss = 'binary_crossentropy',
                        metrics = METRICS)

In [ ]:
np.random.seed(1234)
model_cnn_bilstm.fit(x = data.train['X'], y = data.train['y'],
                  validation_data = (data.valid['X'], data.valid['y']),
                      sample_weight = data.train['w'],
                                    epochs = 15, batch_size =512)

In [ ]:
model_cnn_bilstm.evaluate(x = data.test['X'], y = data.test['y'])

# Cross Validation

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
np.random.seed(1234)
kf = KFold(n_splits=5, random_state=42, shuffle=True)

fold_no = 1       # counter for every K fold in cross validation proces
auc_per_fold = [] # for capturing auc in every fold

In [ ]:
# Making a DataFrame from the train samples
# for CrossValidation
X = np.concatenate([data.train["X"], data.test["X"], data.valid["X"]], axis = 0)
Y = np.concatenate([data.train["y"], data.test["y"], data.valid["y"]], axis = 0)

In [ ]:
Input_Shape = X.shape[1:]

# CV for book model

In [ ]:
for train, test in kf.split(X,Y):
    
    # We have to define model in the loop, so we would have 
    model_book = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters = 30, 
                           kernel_size = 10,
                          activation = 'relu',
                          padding = 'same',
                          input_shape = Input_Shape),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Conv1D(filters = 30, 
                           kernel_size = 10,
                          activation = 'relu',
                          padding = 'same'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Conv1D(filters = 30, 
                           kernel_size = 10,
                          activation = 'relu',
                          padding = 'same'),
        tf.keras.layers.Dropout(0.5),
    
        tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
        ])
    model_book.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3),
                        loss = 'binary_crossentropy',
                        metrics = METRICS)
    
    history = model_book.fit(X[train], Y[train],
                            epochs = 20, batch_size = BATCH_SIZE)
    
    scores = model_book.evaluate(X[test], Y[test],verbose = 0)
    
    auc_per_fold.append(scores[1])
    
    fold_no +=1

In [ ]:
auc_per_fold

# CV for 1D CNN

In [ ]:
for train, test in kf.split(X,Y):
    
    # We have to define model in the loop, so we would have 
    model_cnn = tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(256, 
                                kernel_size = 3,
                                padding = "valid", 
                                activation = "relu",
                                input_shape=Input_Shape),    
            tf.keras.layers.MaxPooling1D(3),
            tf.keras.layers.Conv1D(256, 
                                kernel_size = 3,
                                padding = "valid", 
                                activation = "relu"),    
            tf.keras.layers.MaxPooling1D(3),
            tf.keras.layers.Dropout(0.25),


            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation = 'relu'),
            tf.keras.layers.Dense(64, activation = 'relu'),
            tf.keras.layers.Dense(64),
            tf.keras.layers.Dropout(0.2),
#             tf.keras.layers.Dense(32),
            tf.keras.layers.Dense(16),
            tf.keras.layers.Dense(1,activation='sigmoid')
            ])
    
    opt_cnn = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    model_cnn.compile(optimizer = opt_cnn,
                 loss = 'binary_crossentropy',
                 metrics = METRICS)
    
    history = model_cnn.fit(X[train], Y[train],
                            epochs = 10, batch_size = 128)
    
    scores = model_cnn.evaluate(X[test], Y[test],verbose = 0)
    
    auc_per_fold.append(scores[1])
    
    fold_no +=1

In [ ]:
auc_per_fold

# 1D CNN + BiLSTM

In [ ]:
for train, test in kf.split(X,Y):
    
    # We have to define model in the loop, so we would have 
    model_cnn_bilstm = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(256, 
                          kernel_size = 3, 
                          input_shape = Input_Shape),
    tf.keras.layers.MaxPooling1D(3),
    tf.keras.layers.Conv1D(256, 
                          kernel_size = 3),
    tf.keras.layers.MaxPooling1D(3),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
        ])
    
    opt_cnn = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    model_cnn_bilstm.compile(optimizer = opt_cnn,
                 loss = 'binary_crossentropy',
                 metrics = METRICS)
    
    history = model_cnn_bilstm.fit(X[train], Y[train],
                            epochs = 15, batch_size = 128)
    
    scores = model_cnn_bilstm.evaluate(X[test], Y[test],verbose = 0)
    
    auc_per_fold.append(scores[1])
    
    fold_no +=1

In [ ]:
auc_per_fold